In [28]:
import pandas as pd 
import numpy as np 
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import math
import datetime
from pandas import DataFrame
from pandas import concat
from numpy import concatenate

import tensorflow as tf
from dateutil import parser
import matplotlib.dates as mdates
import matplotlib.ticker as plticker

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [29]:
df = pd.read_csv('Crypto_Tweets.csv')
print(df.shape)
df = df.sort_values(by = 'Date')
df.columns = df.columns.str.replace('Date', 'datetime')

(34072, 8)


In [30]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

df = df.sort_values(by = 'datetime')
df['Tweet'] = [clean_tweet(tweet) for tweet in df['Tweet']]

In [31]:
analyzer = SentimentIntensityAnalyzer()
compound = []

df['Overall'] = df['Tweet'].apply(lambda Tweet: analyzer.polarity_scores(Tweet))
df['Postive'] = df['Overall'].apply(lambda score_dict: score_dict['pos'])
df['Negative'] = df['Overall'].apply(lambda score_dict: score_dict['neg'])
df['Neutral'] = df['Overall'].apply(lambda score_dict: score_dict['neu'])
df = df.drop('Overall', axis=1)

for i,s in enumerate(tqdm(df['Tweet'])):
    vs = analyzer.polarity_scores(s)
    compound.append(vs["compound"])
df["Compound"] = compound
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 34072/34072 [00:06<00:00, 5189.66it/s]


,datetime,ID,Tweet,Username,Location,Favorites,Retweets,Followers,Postive,Negative,Neutral,Compound
76,2022-01-01 00:00:50+00:00,1477067252784586756,Fuck Off 500 nft eth giveaway Just RT and Foll...,pixelpartynft,LuckyBog 😎 TheMetaDealer,1157,1394,23343,0.259,0.133,0.608,0.5106
38,2022-01-01 00:00:52+00:00,1477067263031271426,Bitcoin has started a new yearly candle,intocryptoverse,NaN,3391,283,681517,0.000,0.000,1.000,0.0000
37,2022-01-01 00:03:09+00:00,1477067835503435782,Bitcoin s 2021 yearly candle in the books Than...,WClementeIII,Newsletter👉🏽,4956,399,656584,0.441,0.000,0.559,0.9485
36,2022-01-01 00:42:43+00:00,1477077795108835329,2021 is in the rearview mirror Let s Go Bitcoin,Bitcoin,Worldwide,2903,694,5440179,0.000,0.000,1.000,0.0000
104,2022-01-01 00:46:41+00:00,1477078793810501641,FOLLOW RT AND LIKE THIS TWEET FOR THE CHANCE T...,TitanXBT,NaN,337,252,43990,0.326,0.000,0.674,0.9022


In [32]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['datetime'] = df['datetime'].dt.floor('h')
df['datetime'] = df['datetime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour))
df.head()

,datetime,ID,Tweet,Username,Location,Favorites,Retweets,Followers,Postive,Negative,Neutral,Compound
76,2022-01-01,1477067252784586756,Fuck Off 500 nft eth giveaway Just RT and Foll...,pixelpartynft,LuckyBog 😎 TheMetaDealer,1157,1394,23343,0.259,0.133,0.608,0.5106
38,2022-01-01,1477067263031271426,Bitcoin has started a new yearly candle,intocryptoverse,NaN,3391,283,681517,0.000,0.000,1.000,0.0000
37,2022-01-01,1477067835503435782,Bitcoin s 2021 yearly candle in the books Than...,WClementeIII,Newsletter👉🏽,4956,399,656584,0.441,0.000,0.559,0.9485
36,2022-01-01,1477077795108835329,2021 is in the rearview mirror Let s Go Bitcoin,Bitcoin,Worldwide,2903,694,5440179,0.000,0.000,1.000,0.0000
104,2022-01-01,1477078793810501641,FOLLOW RT AND LIKE THIS TWEET FOR THE CHANCE T...,TitanXBT,NaN,337,252,43990,0.326,0.000,0.674,0.9022


In [33]:
df.set_index(['datetime'])

,ID,Tweet,Username,Location,Favorites,Retweets,Followers,Postive,Negative,Neutral,Compound
datetime,,,,,,,,,,,
2022-01-01 00:00:00,1477067252784586756,Fuck Off 500 nft eth giveaway Just RT and Foll...,pixelpartynft,LuckyBog 😎 TheMetaDealer,1157,1394,23343,0.259,0.133,0.608,0.5106
2022-01-01 00:00:00,1477067263031271426,Bitcoin has started a new yearly candle,intocryptoverse,NaN,3391,283,681517,0.000,0.000,1.000,0.0000
2022-01-01 00:00:00,1477067835503435782,Bitcoin s 2021 yearly candle in the books Than...,WClementeIII,Newsletter👉🏽,4956,399,656584,0.441,0.000,0.559,0.9485
2022-01-01 00:00:00,1477077795108835329,2021 is in the rearview mirror Let s Go Bitcoin,Bitcoin,Worldwide,2903,694,5440179,0.000,0.000,1.000,0.0000
2022-01-01 00:00:00,1477078793810501641,FOLLOW RT AND LIKE THIS TWEET FOR THE CHANCE T...,TitanXBT,NaN,337,252,43990,0.326,0.000,0.674,0.9022
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 22:00:00,1531771109602799617,Remember that you should always test your dapp...,VitalikButerin,Earth,9346,1113,4155666,0.169,0.000,0.831,0.7115
2022-05-31 23:00:00,1531775365219520512,Discord Opening Soon Interact with this tweet ...,ZoomersNFT,Zoomers District,1258,1001,35724,0.371,0.102,0.528,0.7983
2022-05-31 23:00:00,1531777170808479744,500 in ETH 1 Winner Follow Retweet 24h Good lu...,cryp_toplayer,Island boy,647,641,69776,0.581,0.000,0.419,0.8658


In [34]:
df_price = pd.read_csv("../data/flattened_data.csv")
df_price.set_index(['datetime'])

,1INCHBUSD_open,1INCHBUSD_high,1INCHBUSD_low,1INCHBUSD_close,1INCHBUSD_volume,1INCHUSD_open,1INCHUSD_high,1INCHUSD_low,1INCHUSD_close,1INCHUSD_volume,...,ZRXBUSD_open,ZRXBUSD_high,ZRXBUSD_low,ZRXBUSD_close,ZRXBUSD_volume,ZRXUSD_open,ZRXUSD_high,ZRXUSD_low,ZRXUSD_close,ZRXUSD_volume
datetime,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,2.410,2.450,2.406,2.429,27034.1,2.390,2.420,2.390,2.420,47726.30,...,0.8225,0.8375,0.8225,0.8349,2957.0,0.803274,0.815819,0.803274,0.815819,124094.48027
2022-01-01 01:00:00,2.434,2.441,2.421,2.424,15641.5,2.420,2.440,2.410,2.440,109643.15,...,0.8367,0.8367,0.8262,0.8262,7690.0,0.815761,0.818488,0.812232,0.816939,430003.38704
2022-01-01 02:00:00,2.423,2.444,2.418,2.434,10500.5,2.430,2.450,2.430,2.440,22534.85,...,0.8249,0.8305,0.8249,0.8296,1352.0,0.817332,0.841967,0.816370,0.825341,545121.67633
2022-01-01 03:00:00,2.433,2.449,2.433,2.435,11621.9,2.430,2.450,2.430,2.430,16066.99,...,0.8310,0.8327,0.8271,0.8271,5814.0,0.825479,0.826189,0.819242,0.821205,174704.12975
2022-01-01 04:00:00,2.436,2.454,2.415,2.428,18122.9,2.420,2.430,2.400,2.400,35143.16,...,0.8262,0.8326,0.8239,0.8326,4731.0,0.822383,0.824401,0.814999,0.821184,109516.28991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,0.967,0.972,0.953,0.964,62571.4,0.966,0.972,0.966,0.969,34241.32,...,0.4214,0.4370,0.4193,0.4309,118348.0,0.415042,0.417221,0.412115,0.414259,146124.40386
2022-05-31 20:00:00,0.963,0.965,0.960,0.965,19140.8,0.969,0.970,0.966,0.968,5846.26,...,0.4270,0.4432,0.4252,0.4308,148208.0,0.414923,0.421796,0.414923,0.421796,48940.19297
2022-05-31 21:00:00,0.964,0.964,0.958,0.961,15210.7,0.969,0.970,0.967,0.969,5068.87,...,0.4295,0.4445,0.4295,0.4410,149321.0,0.421837,0.421837,0.416822,0.417250,35316.14544


In [45]:
Final_df = df_price.merge(df, left_index=True, right_index=True, how='inner')
Final_df.to_csv('crypto_data_tweet_final.csv')

In [47]:
cor = Final_df.corr()
cor

,1INCHBUSD_open,1INCHBUSD_high,1INCHBUSD_low,1INCHBUSD_close,1INCHBUSD_volume,1INCHUSD_open,1INCHUSD_high,1INCHUSD_low,1INCHUSD_close,1INCHUSD_volume,...,ZRXUSD_close,ZRXUSD_volume,ID,Favorites,Retweets,Followers,Postive,Negative,Neutral,Compound
1INCHBUSD_open,1.000000,0.999437,0.999356,0.999008,-0.031122,0.995113,0.995432,0.995760,0.996111,0.099593,...,0.663270,-0.073886,-0.824899,0.003038,-0.000568,-0.002891,0.032670,0.004041,-0.032557,0.021726
1INCHBUSD_high,0.999437,1.000000,0.999078,0.999427,-0.015538,0.994896,0.995256,0.995423,0.995824,0.102589,...,0.661772,-0.071375,-0.825682,0.002319,-0.001385,-0.002391,0.033628,0.002788,-0.032773,0.023052
1INCHBUSD_low,0.999356,0.999078,1.000000,0.999524,-0.044061,0.993956,0.994187,0.994750,0.995012,0.094668,...,0.663434,-0.075210,-0.823934,0.002908,-0.000643,-0.003134,0.031508,0.004156,-0.031540,0.020748
1INCHBUSD_close,0.999008,0.999427,0.999524,1.000000,-0.029796,0.994163,0.994460,0.994817,0.995136,0.098299,...,0.662219,-0.073637,-0.824930,0.002254,-0.001321,-0.002665,0.032729,0.002871,-0.031982,0.022461
1INCHBUSD_volume,-0.031122,-0.015538,-0.044061,-0.029796,1.000000,-0.028261,-0.025077,-0.031183,-0.028169,0.188184,...,-0.008950,0.075730,0.101980,-0.013863,-0.013975,-0.011550,0.026634,-0.048419,0.001304,0.038306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Followers,-0.002891,-0.002391,-0.003134,-0.002665,-0.011550,-0.002462,-0.002306,-0.002396,-0.001923,-0.009298,...,0.004173,0.000686,0.001660,0.439052,0.280741,1.000000,-0.051766,-0.020577,0.059160,-0.053171
Postive,0.032670,0.033628,0.031508,0.032729,0.026634,0.035550,0.036481,0.034540,0.035146,0.056584,...,0.021117,0.017513,-0.041591,-0.048795,0.026412,-0.051766,1.000000,-0.152661,-0.847038,0.768035
Negative,0.004041,0.002788,0.004156,0.002871,-0.048419,0.003850,0.003380,0.004148,0.004164,-0.018563,...,-0.011883,-0.019743,-0.020376,0.037786,-0.014497,-0.020577,-0.152661,1.000000,-0.395989,-0.521582
Neutral,-0.032557,-0.032773,-0.031540,-0.031982,0.001304,-0.035132,-0.035743,-0.034354,-0.034924,-0.042595,...,-0.013246,-0.005658,0.049606,0.025033,-0.016689,0.059160,-0.847038,-0.395989,1.000000,-0.433051


In [48]:
Final_df.describe()

,1INCHBUSD_open,1INCHBUSD_high,1INCHBUSD_low,1INCHBUSD_close,1INCHBUSD_volume,1INCHUSD_open,1INCHUSD_high,1INCHUSD_low,1INCHUSD_close,1INCHUSD_volume,...,ZRXUSD_close,ZRXUSD_volume,ID,Favorites,Retweets,Followers,Postive,Negative,Neutral,Compound
count,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3624.000000,3.624000e+03,...,3624.000000,3.624000e+03,3.624000e+03,3624.000000,3624.000000,3.624000e+03,3624.000000,3624.000000,3624.000000,3624.000000
mean,1.559733,1.571621,1.546958,1.559310,27272.911393,1.561863,1.574350,1.548397,1.561491,3.205842e+04,...,0.627513,2.862425e+05,1.481011e+18,2953.575331,822.683775,7.365026e+05,0.143323,0.040179,0.816501,0.300684
std,0.402551,0.404702,0.399688,0.402413,36862.298094,0.403310,0.406142,0.399686,0.403193,4.997809e+04,...,0.147214,4.961576e+05,2.209589e+15,9054.021616,1858.147366,2.789779e+06,0.139252,0.080603,0.149866,0.441002
min,0.739000,0.783000,0.642000,0.743000,739.700000,0.746000,0.782000,0.672000,0.746000,1.091100e+02,...,0.305072,8.626905e+03,1.477067e+18,70.000000,62.000000,4.100000e+01,0.000000,0.000000,0.000000,-0.984300
25%,1.317750,1.327000,1.307000,1.317000,8865.600000,1.318000,1.327000,1.308000,1.318000,7.462565e+03,...,0.513416,7.439558e+04,1.479131e+18,895.750000,303.750000,3.337950e+04,0.000000,0.000000,0.714000,0.000000
50%,1.540500,1.551000,1.530000,1.540500,16178.700000,1.541000,1.552000,1.530000,1.541000,1.816287e+04,...,0.613962,1.501265e+05,1.481029e+18,1745.500000,468.000000,1.948830e+05,0.126000,0.000000,0.826000,0.318200
75%,1.763000,1.775250,1.749250,1.762250,30926.350000,1.767000,1.780000,1.750000,1.764250,3.768640e+04,...,0.736851,3.276185e+05,1.482913e+18,3186.000000,788.000000,7.232918e+05,0.235000,0.059000,0.958000,0.690800
max,2.620000,2.663000,2.598000,2.620000,576755.000000,2.620000,2.660000,2.600000,2.620000,1.006912e+06,...,1.114070,1.034056e+07,1.485039e+18,382511.000000,49799.000000,1.026654e+08,0.756000,1.000000,1.000000,0.983300
